## Instructions

Create a project with code that solves the following questions. Please generate a repository in GitHub with the code and make all necessary commits. The use of any AI tool is forbidden.

1. (4 points) Generate a pipeline for processing a text. The pipeline will have the following porcesses: Sentence splitter, word tokenization, conver acronims with dots to acronims whitout dots with regex (U.S.A. to USA), remove stopwords, lematizer. You must do each step in pure python and with a NLP library of your choice.

2. (4 points) Create an application with user interface that call a local LLM to solve any problem you choose. Modify the system prompt to better solve the problem.

3. (2 points) Integrate in the application of the previous question with a speech model for speech to text and text to speech. The model should be from Huging Face.

Please upload the code files and a link to the github repository. The assesment will be with a one on one defense. 

## Part 1

Genarate a pipeline for processing a text.

the text I will be using to complete this task is as follows:

In [40]:
text = "The president of the U.S.A., Donald Trump, is 1.9m high and 78 years old. Forbes Magazine has assessed his wealth, currently estimating it at $5.5 billion as of mid-February 2025."

### Imports

In [41]:
import nltk
import re
from num2words import num2words

Using NLTK to split sentences

In [42]:
sentences = nltk.sent_tokenize(text)

print(sentences)

['The president of the U.S.A., Donald Trump, is 1.9m high and 78 years old.', 'Forbes Magazine has assessed his wealth, currently estimating it at $5.5 billion as of mid-February 2025.']


Using NLTK for word tokenization

In [43]:
words = nltk.word_tokenize(text)

print(words)

['The', 'president', 'of', 'the', 'U.S.A.', ',', 'Donald', 'Trump', ',', 'is', '1.9m', 'high', 'and', '78', 'years', 'old', '.', 'Forbes', 'Magazine', 'has', 'assessed', 'his', 'wealth', ',', 'currently', 'estimating', 'it', 'at', '$', '5.5', 'billion', 'as', 'of', 'mid-February', '2025', '.']


Convert acronyms with regex

In [44]:
# For U:S:A.
text = re.sub(r'U\.S\.A\.', 'USA', text)

# For numbers of a height and money

text = re.sub(r'(\d+\.\d+)m', lambda x: f"{int(float(x.group(1)) * 100)} centimeters", text)

def decimal_to_words(match):
    number = match.group(1)  # Extract the decimal number ("5.5")
    integer_part, decimal_part = number.split(".")  # Split into whole and decimal parts
    return f"{num2words(int(integer_part))} point {num2words(int(decimal_part))} billion"

text = re.sub(r'\$(\d+\.\d+)\sbillion', decimal_to_words, text)

print(text)

The president of the USA, Donald Trump, is 190 centimeters high and 78 years old. Forbes Magazine has assessed his wealth, currently estimating it at five point five billion as of mid-February 2025.


remove stopwords using nltk

In [45]:
# Convert to lower case (except proper nouns)
def process_text(text):
    proper_nouns = ["USA", "Donald Trump", "Forbes Magazine", "mid-February"]
    
    for proper in proper_nouns:
        text = re.sub(r'\b' + re.escape(proper) + r'\b', proper.replace(" ", "_"), text)

    words = text.split()
    processed_words = []

    for word in words:
        if "_" in word or word.isupper() or word == "mid-February":  
            processed_words.append(word)
        else:
            processed_words.append(word.casefold())

    return " ".join(processed_words)

processed_text = process_text(text)
print(processed_text)

the president of the USA, Donald_Trump, is 190 centimeters high and 78 years old. Forbes_Magazine has assessed his wealth, currently estimating it at five point five billion as of mid-February 2025.


In [46]:
# Tokenize the processed text
processed_words = nltk.word_tokenize(processed_text)
processed_sentences = nltk.sent_tokenize(processed_text)
print(processed_sentences)
print(processed_words)

['the president of the USA, Donald_Trump, is 190 centimeters high and 78 years old.', 'Forbes_Magazine has assessed his wealth, currently estimating it at five point five billion as of mid-February 2025.']
['the', 'president', 'of', 'the', 'USA', ',', 'Donald_Trump', ',', 'is', '190', 'centimeters', 'high', 'and', '78', 'years', 'old', '.', 'Forbes_Magazine', 'has', 'assessed', 'his', 'wealth', ',', 'currently', 'estimating', 'it', 'at', 'five', 'point', 'five', 'billion', 'as', 'of', 'mid-February', '2025', '.']


In [47]:
# nltk.download('stopwords')

def removestopwords(tokens):
    stop_words = set(nltk.corpus.stopwords.words("english"))  # Get the stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

filtered_words = removestopwords(processed_words)

print(filtered_words)

['president', 'USA', ',', 'Donald_Trump', ',', '190', 'centimeters', 'high', '78', 'years', 'old', '.', 'Forbes_Magazine', 'assessed', 'wealth', ',', 'currently', 'estimating', 'five', 'point', 'five', 'billion', 'mid-February', '2025', '.']


Lemmatizer

In [ ]:
# Imports
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

#nltk.download()
lemmatizer = WordNetLemmatizer()



def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
pos_tag = nltk.pos_tag(filtered_words)

lemmatized_sentences = []
for token, tag in pos_tag:
    wordnet_pos = get_wordnet_pos(tag) or wordnet.NOUN # if no pos tag found
    lemmatized_sentences.append(lemmatizer.lemmatize(filtered_words), pos=wordnet_pos)

TypeError: unhashable type: 'list'

## Part 2

Create an application with user interface that call a local LLM to solve any problem you choose. Modify the system prompt to better solve the problem.